In [31]:
import numpy as np
import pandas as pd
import csv
from scipy.sparse import csc_matrix

#constants defining the dimensions of our User Rating Matrix (URM)
MAX_BID = 31 #MAX Bolt ID
MAX_TID = 11 #MAX Termékcsoport
K=2


def readUrm(mit):
	urm = np.zeros(shape=(MAX_BID,MAX_TID), dtype=np.float32)
	with open(mit, 'rb') as trainFile:
		urmReader = csv.reader(trainFile, delimiter=';')
		for row in urmReader:
			urm[int(row[0]), int(row[1])] = float(row[2])

	return csc_matrix(urm, dtype=np.float32)

In [32]:
data=readUrm('SVDTrain.csv')

In [33]:
data.todense()

matrix([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   1.66000000e+04,   2.69000000e+04,
           7.40000000e+03,   1.30000000e+03,   6.78000000e+04,
           9.00000000e+03,   4.03000000e+04,   2.40000000e+04,
           8.00000000e+03,   2.41800000e+05],
        [  0.00000000e+00,   4.73000000e+04,   6.68000000e+04,
           3.47000000e+04,   8.20000000e+03,   2.37500000e+05,
           4.26000000e+04,   1.60900000e+05,   2.92000000e+04,
           3.60000000e+04,   4.56100000e+05],
        [  0.00000000e+00,   3.17000000e+04,   6.60000000e+03,
           2.70000000e+03,   0.00000000e+00,   4.94000000e+04,
           5.50000000e+03,   2.21000000e+04,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   5.30000000e+0

In [34]:
def readUsersTest():
	uTest = dict()
	with open("SVDBoltok.csv", 'rb') as testFile:
		testReader = csv.reader(testFile, delimiter=';')
		i=0
		for row in testReader:
			uTest[int(row[0])] = list()

	return uTest

In [35]:
def getMoviesSeen():
	moviesSeen = dict()
	with open("SVDTrain.csv", 'rb') as trainFile:
		urmReader = csv.reader(trainFile, delimiter=';')
		for row in urmReader:
			try:
				moviesSeen[int(row[0])].append(int(row[1]))
			except:
				moviesSeen[int(row[0])] = list()
				moviesSeen[int(row[0])].append(int(row[1]))

	return moviesSeen

In [36]:
import math as mt
import csv
from sparsesvd import sparsesvd

def computeSVD(urm, K):
	U, s, Vt = sparsesvd(urm, K)

	dim = (len(s), len(s))
	S = np.zeros(dim, dtype=np.float32)
	for i in range(0, len(s)):
		S[i,i] = mt.sqrt(s[i])

	U = csc_matrix(np.transpose(U), dtype=np.float32)
	S = csc_matrix(S, dtype=np.float32)
	Vt = csc_matrix(Vt, dtype=np.float32)

	return U, S, Vt	

In [37]:
from scipy.sparse.linalg import * #used for matrix multiplication

def computeEstimatedRatings2(urm, U, S, Vt, uTest, moviesSeen, K, test):
	rightTerm = S*Vt 

	estimatedRatings = np.zeros(shape=(MAX_BID, MAX_TID), dtype=np.float16)
	for userTest in uTest:
		prod = U[userTest, :]*rightTerm

		#we convert the vector to dense format in order to get the indices of the movies with the best estimated ratings 
		estimatedRatings[userTest, :] = prod.todense()
		recom = (-estimatedRatings[userTest, :]).argsort()
		for r in recom:
			if r not in moviesSeen[userTest]:
				uTest[userTest].append(r)


	return uTest

In [38]:
from scipy.sparse.linalg import * #used for matrix multiplication

def computeEstimatedRatings(urm, U, S, Vt, uTest, moviesSeen, K, test):
	rightTerm = S*Vt 

	estimatedRatings = np.zeros(shape=(MAX_BID, MAX_TID), dtype=np.float16)
	for userTest in uTest:
		prod = U[userTest, :]*rightTerm

		#we convert the vector to dense format in order to get the indices of the movies with the best estimated ratings 
		estimatedRatings[userTest, :] = prod.todense()
		recom = (-estimatedRatings[userTest, :])
		for r in recom:
			if r not in moviesSeen[userTest]:
				uTest[userTest].append(r)

	return uTest

In [71]:
def main():
    #értékek
    K = 20
    urm = readUrm('SVDTrain.csv')
    U, S, Vt = computeSVD(urm,K)
    uTest = readUsersTest()
    moviesSeen = getMoviesSeen()
    uTest = computeEstimatedRatings(urm, U, S, Vt, uTest, moviesSeen, K, True)
    #indexek
    Ux, Sx, Vtx = computeSVD(urm,K)
    uTestx = readUsersTest()
    moviesSeen = getMoviesSeen()
    uTestx = computeEstimatedRatings2(urm, Ux, Sx, Vtx, uTestx, moviesSeen, K, True)
    
    teljes=pd.read_excel("csoportNOTnorm.xlsx").as_matrix()
    
    for key in uTest.keys():
        for i in uTestx[key]:
            if(i!=0):
                print("Bolt: %d , Termék család:%d érték:%.2f eredeti:%d" % (key,i,uTest[key][i],teljes[key-1][i-1]))

In [72]:
main()

[[  16600   26900    7400    1300   67800    9000   40300   24000    8000
   241800]
 [  47300   66800   34700    8200  237500   42600  160900   29200   36000
   456100]
 [  31700    6600    2700       0   49400    5500   22100   13400    3300
   487300]
 [   3300    5300       0       0   13800     600   19500   10800     400
    34000]
 [   6000   21200    1600     600   18800     400   21500    9500    2700
   168100]
 [  13200    5100    1400       0   13900   10500   28500    7800     700
    25100]
 [  14800   18300    5000       0   46300    1800   37500   15900   13300
   103300]
 [   2900   33100    4800    2400   50500    6700   33100   25000    8500
   113200]
 [   -400   28700       0    1100   61000   15000   49300    9600    3300
   -57200]
 [   2600   13500   10100     800   39400    -600   29300   20800    7100
    39300]
 [   5000    2500    -500       0    7000    1200   21000   12600     600
    25800]
 [   9600   19600     500       0   41800    1400   12700    9000